In [9]:
# Import the Altair library
import pandas as pd
import altair as alt
import numpy as np
import csv
# work-around to let Altair handle larger data sets
alt.data_transformers.enable('json')


# Reference to data
france = pd.read_csv('./src/assets/dpt2020.csv', sep=',')

print(france.columns)

Index(['sexe', 'preusuel', 'annais', 'dpt', 'nombre'], dtype='object')


In [11]:
"""
def optimize(csv_data):
    if csv_data is None:
        raise Exception("CSV was not loaded when optimize was called")
    else:
        print("Optimizing dataset")

    filteredByNameByYear = {}
     
    for _,row in csv_data.iterrows():
        annais = row['annais']
        preusuel = row['preusuel']
        if preusuel not in filteredByNameByYear:
            filteredByNameByYear[preusuel] = {}
        if row.annais not in filteredByNameByYear[preusuel]:
            filteredByNameByYear[preusuel][annais] = []
        filteredByNameByYear[preusuel][annais].append(row)

    print("Optimization done")
    return filteredByNameByYear

opti_france = optimize(france)
"""


'\ndef optimize(csv_data):\n    if csv_data is None:\n        raise Exception("CSV was not loaded when optimize was called")\n    else:\n        print("Optimizing dataset")\n\n    filteredByNameByYear = {}\n     \n    for _,row in csv_data.iterrows():\n        annais = row[\'annais\']\n        preusuel = row[\'preusuel\']\n        if preusuel not in filteredByNameByYear:\n            filteredByNameByYear[preusuel] = {}\n        if row.annais not in filteredByNameByYear[preusuel]:\n            filteredByNameByYear[preusuel][annais] = []\n        filteredByNameByYear[preusuel][annais].append(row)\n\n    print("Optimization done")\n    return filteredByNameByYear\n\nopti_france = optimize(france)\n'

In [12]:
#print(opti_france['CAMILLE']['1900'])

In [14]:
# Display the evolution of the proportion of male 'CLAUDE' in France over the years

"""
INPUT : csv with 'male' : nb_of_male, 'female' : nb_of_female, 'total' : total_number

"""

selected_name = 'CLAUDE'

#
#table = opti_france[selected_name]



In [ ]:
def optimize_with_csv(csv_data):
    if csv_data is None:
        raise Exception("CSV was not loaded when optimize was called")
    else:
        print("Optimizing dataset")
    
    #create an empty CSV file
    with open('optimized.csv', 'w', newline='') as csvfile:
        fieldnames = ['previsuel', 'annee', 'male', 'female', 'total']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    filteredByNameByYear = {}
     
    for _,row in csv_data.iterrows():
        annais = row['annais']
        preusuel = row['preusuel']
        if preusuel not in filteredByNameByYear:
            filteredByNameByYear[preusuel] = {}
        if row.annais not in filteredByNameByYear[preusuel]:
            filteredByNameByYear[preusuel][annais] = []
        
        filteredByNameByYear[preusuel][annais].append(row)

        #write the data in the CSV file
        with open('optimized.csv', 'a', newline='') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writerow({'previsuel': row['previsuel'], 'annee': row['annee'], 'male': row['male'], 'female': row['female'], 'total': row['total']})

    print("Optimization done")
    return filteredByNameByYear


opti_france = optimize_with_csv(france)


Optimizing dataset


KeyError: 'previsuel'

In [ ]:
# Display the evolution of the proportion of male 'CLAUDE' in France over the years

selected_name = 'CLAUDE'

filtered_name = france[(france['preusuel'] == selected_name) & (france['annais'] != 'XXXX')]
search_input = alt.selection_point(
    fields = ["preusuel"],
    value = selected_name,
    bind = alt.binding(
        input="search",
        placeholder="Name",
        name='Search',
    )
)
filtered_france =  france[(france['annais'] != 'XXXX')]

gender_proportion_graph = alt.Chart(opti_france).transform_filter(
    search_input
).transform_aggregate(
    sum_selection='sum(nombre)',
    groupby=['annais', 'sexe']
).transform_joinaggregate(
    total_sum='sum(sum_selection)',
    groupby=['annais']
).transform_calculate(
    proportion='datum.sum_selection / datum.total_sum*100',
    sex_label='datum.sexe === 1 ? "Male" : "Female"'
).mark_bar().encode(
    x=alt.X('annais:O', title='Year', scale=alt.Scale(domain=[str(year) for year in range(1900, 2021)])),
    y=alt.Y('proportion:Q', title='Percentage by sex'),
    color=alt.Color('sex_label:N',scale=alt.Scale(domain=['Male', 'Female'],range=['red', 'blue']),legend=alt.Legend(title='Sexe')),
    tooltip=[
        alt.Tooltip('annais', title='Year'),
        alt.Tooltip('proportion:Q', title='Percentage'),
        alt.Tooltip("sum_selection:Q", title="Number of births"),
        alt.Tooltip("sex_label:N", title="Sex")
    ]
).properties(
    width=1000,
    height=400,
    title="Births for searched name"
).add_params(
    search_input
)
### this is a bar chart
gender_proportion_graph

KeyboardInterrupt: 

In [ ]:
selected_name = 'CLAUDE'

filtered_F_selected_name = france[(france['sexe'] == 2) & (
    france['preusuel'] == 'CLAUDE') & (france['annais'] != 'XXXX')]
filtered_M_selected_name = france[(france['sexe'] == 1) & (
    france['preusuel'] == 'CLAUDE')& (france['annais'] != 'XXXX')]

print(sum(filtered_F_selected_name['nombre']))

54031


In [ ]:
# Display the evolution of the proportion of male 'CLAUDE' in France over the years

selected_name = 'MARIE'
filtered_name = france[(france['preusuel'] == selected_name) & (france['annais'] != 'XXXX')]

gender_proportion_graph = alt.Chart(filtered_name).transform_aggregate(
    sum_selection='sum(nombre)',
    groupby=['annais', 'sexe']
).transform_calculate(
    sex_label='datum.sexe === 1 ? "Male" : "Female"'
).mark_circle().encode(
    x=alt.X('annais:O', title='Year', scale=alt.Scale(domain=[str(year) for year in range(1900, 2021)])),
    y=alt.Y('sum_selection:Q', title='Percentage by sex'),
    color=alt.Color('sex_label:N',scale=alt.Scale(domain=['Male', 'Female'],range=['red', 'blue']),legend=alt.Legend(title='Sexe')),
    tooltip=[
        alt.Tooltip('annais', title='Year'),
        alt.Tooltip("sum_selection:Q", title="Number of births"),
        alt.Tooltip("sex_label:N", title="Sex")
    ]

).properties(
    width=1000,
    height=500,
    title="Births for the name "+selected_name
)


###this is a scatter plot
gender_proportion_graph

alt.Chart(...)

Le but c'est de trouver les proportions de prénoms selon les genres, par année. Donc j'essaie de faire un sorte d'avoir le nombre dans 2 tables différentes pour pouvoir faire le rapport.



In [ ]:
# Display the evolution of the proportion of male 'CLAUDE' in France over the years

selected_name = 'CLAUDE'

filtered_name = france[(france['preusuel'] == selected_name) & (france['annais'] != 'XXXX')]
search_input = alt.selection_point(
    fields = ["preusuel"],
    value = selected_name,
    bind = alt.binding(
        input="search",
        placeholder="Name",
        name='Search',
    )
)
filtered_france =  france[(france['annais'] != 'XXXX')]

gender_proportion_graph = alt.Chart(filtered_france).transform_filter(
    search_input
).transform_aggregate(
    sum_selection='sum(nombre)',
    groupby=['annais', 'sexe']
).transform_joinaggregate(
    total_sum='sum(sum_selection)',
    groupby=['annais']
).transform_calculate(
    proportion='datum.sum_selection / datum.total_sum*100',
    sex_label='datum.sexe === 1 ? "Male" : "Female"'
).mark_bar().encode(
    x=alt.X('annais:O', title='Year', scale=alt.Scale(domain=[str(year) for year in range(1900, 2021)])),
    y=alt.Y('proportion:Q', title='Percentage by sex'),
    color=alt.Color('sex_label:N',scale=alt.Scale(domain=['Male', 'Female'],range=['red', 'blue']),legend=alt.Legend(title='Sexe')),
    tooltip=[
        alt.Tooltip('annais', title='Year'),
        alt.Tooltip('proportion:Q', title='Percentage'),
        alt.Tooltip("sum_selection:Q", title="Number of births"),
        alt.Tooltip("sex_label:N", title="Sex")
    ]
).properties(
    width=1000,
    height=400,
    title="Births for searched name"
).add_params(
    search_input
)
### this is a bar chart
gender_proportion_graph

In [ ]:
# Display the evolution of the proportion of male 'CLAUDE' in France over the years

selected_name = 'CAMILLE'
filtered_name = france[(france['preusuel'] == selected_name) & (france['annais'] != 'XXXX')]

gender_proportion_graph = alt.Chart(filtered_name).transform_aggregate(
    sum_selection_M='sum(nombre)',
    groupby=['annais', 'sexe']
).transform_joinaggregate(
    total_sum='sum(sum_selection_M)',
    groupby=['annais']
).transform_calculate(
    proportion='datum.sum_selection_M / datum.total_sum*100',
    sex_label='datum.sexe === 1 ? "Male" : "Female"'
).transform_filter(
    (alt.datum.sexe == 1) | (alt.datum.sexe == 2)
).mark_circle().encode(
    x=alt.X('annais:O', title='Year', scale=alt.Scale(domain=[str(year) for year in range(1900, 2021)])),
    y=alt.Y('proportion:Q', title='Percentage by sex'),
    color=alt.Color('sex_label:N',scale=alt.Scale(domain=['Male', 'Female'],range=['red', 'blue']),legend=alt.Legend(title='Sexe')),
    tooltip=[
        alt.Tooltip('annais', title='Year'),
        alt.Tooltip('proportion:Q', title='Percentage'),
        alt.Tooltip("sum_selection_M:Q", title="Number of births"),
        alt.Tooltip("sex_label:N", title="Sex")
    ]
).properties(
    width=1000,
    height=400,
    title="Births for the name "+selected_name
)


###this is a scatter plot
gender_proportion_graph

alt.Chart(...)

alt.Chart(...)